![](../docs/images/itam_logo.png)

M. Sc. Liliana Millán Núñez liliana.millan@itam.mx

Febrero 2020


## AWS with Python

In [3]:
import boto3 

In [4]:
# saber qué versión de Boto estamos ocupando
print(boto3.__version__)

1.12.11


Si tienes tu archivo `~/.aws/credentials` puedes ocupar dentro del objeto `Session`, qué perfil vas a ocupar para conectarte a AWS, también es posible indicar la región y las credenciales asociadas, aunque esto **no** se recomienda, en este caso es mejor poner las llaves en plano en algún archivo `json` en tu computadora y leerlos desde ahí para que **no** subas tus llaves al repositorio!.

In [6]:
ses = boto3.session.Session(profile_name='gabster', region_name='us-west-2')

In [7]:
s3 = ses.resource('s3')

In [13]:
# listar los buckets que están en este perfil y región.
for bucket in s3.buckets.all():
    print(bucket.name)

test-aws-gab


### Creación de un bucket

+ Por default los *buckets* son creados en la región *US East (N. Virginia)* `us-east-1`, necesitarás definir en qué región quieres crear el *bucket*. 
+ Los nombres de buckets **no** pueden llevar guiónes bajos, te marcará que el nombre es inválido, en lugar de los guiones bajos puedes ocupar guiónes altos.

In [12]:
#crear un bucket
bucket_name = 'test-aws-gab'

s3.create_bucket(Bucket=bucket_name, 
                 CreateBucketConfiguration={'LocationConstraint': 'us-west-2'},
                ACL='private')

s3.Bucket(name='test-aws-gab')

### Descargamos el archivo con la base de datos

In [23]:
import csv
import urllib.request as requests
import io

In [31]:
import time
import pandas as pd

In [38]:
start = time.time()
url = 'https://datos.cdmx.gob.mx/explore/dataset/afluencia-diaria-del-metro-cdmx/download/?format=csv&refine.ano=2019&refine.linea=Linea+3&refine.estacion=Potrero&timezone=America/Mexico_City&lang=es&use_labels_for_header=true&csv_separator=%2C'
url_open = requests.urlopen(url)
csvfile = csv.reader(io.StringIO(url_open.read().decode('utf-8')), delimiter=',')
end = time.time()
print(end - start)

0.6902999877929688


In [39]:
for row in csvfile:
    print(', '.join(row))

Fecha, Dia, Mes, Año, Linea, Estacion, Afluencia
2019-01-01, 1, enero, 2019, Linea 3, Potrero, 5448
2019-03-01, 3, enero, 2019, Linea 3, Potrero, 14899
2019-04-01, 4, enero, 2019, Linea 3, Potrero, 16301
2019-01-14, 14, enero, 2019, Linea 3, Potrero, 16773
2019-01-19, 19, enero, 2019, Linea 3, Potrero, 17314
2019-01-22, 22, enero, 2019, Linea 3, Potrero, 16526
2019-01-26, 26, enero, 2019, Linea 3, Potrero, 17629
2019-01-30, 30, enero, 2019, Linea 3, Potrero, 18814
2019-02-02, 2, febrero, 2019, Linea 3, Potrero, 16669
2019-07-02, 7, febrero, 2019, Linea 3, Potrero, 20766
2019-08-02, 8, febrero, 2019, Linea 3, Potrero, 19479
2019-09-02, 9, febrero, 2019, Linea 3, Potrero, 18891
2019-10-02, 10, febrero, 2019, Linea 3, Potrero, 9475
2019-02-13, 13, febrero, 2019, Linea 3, Potrero, 18453
2019-02-06, 2, junio, 2019, Linea 3, Potrero, 10914
2019-10-06, 10, junio, 2019, Linea 3, Potrero, 18215
2019-12-06, 12, junio, 2019, Linea 3, Potrero, 16652
2019-06-24, 24, junio, 2019, Linea 3, Potrero, 1

In [33]:
start = time.time()
url = "https://datos.cdmx.gob.mx/explore/dataset/afluencia-diaria-del-metro-cdmx/download/?format=csv&refine.ano=2014&timezone=America/Mexico_City&lang=es&use_labels_for_header=true&csv_separator=%2C"
df = pd.read_csv(url)
end = time.time()
print(end - start)

36.89885377883911
        Fecha  Dia    Mes   Año    Linea            Estacion  Afluencia
0  2014-01-01    1  enero  2014  Linea 1  Blvd. Puerto Aéreo      17296
1  2014-01-01    1  enero  2014  Linea 1  Isabel la Católica       6701
2  2014-01-01    1  enero  2014  Linea 1      Salto del Agua       7543
3  2014-01-01    1  enero  2014  Linea 1            Tacubaya      10183
4  2014-01-01    1  enero  2014  Linea 1        Observatorio      48294


### Guardar archivos en el bucket

In [55]:
# subiremos un archivo csv de ejemplo: pollution con ~43k registros 
#con observaciones de contaminación del aire en una ciudad de China. 
file_to_upload = '/home/silil/Documents/pollution.csv'
file_name = file_to_upload.split(sep='/')[-1]

# accedemos a client desde el resource
s3.meta.client.upload_file(file_to_upload, bucket_name, file_name)

### Acceder al bucket

In [56]:
# acceder al bucket
my_bucket = s3.Bucket(bucket_name)

for element in my_bucket.objects.all():
    print(element)

s3.ObjectSummary(bucket_name='test-aws-boto', key='pollution.csv')


### Bajar archivos del bucket

In [60]:
path_to_download = '/home/silil/Documents/itam/metodos_gran_escala/data-product-architecture/' + file_name

s3.meta.client.download_file(bucket_name, file_name, path_to_download)

### Borrar archivos del bucket

In [61]:
s3.Object(bucket_name, file_name).delete()

{'ResponseMetadata': {'RequestId': 'F6D1AA9745ADFEBE',
  'HostId': '6SAzJwWvNHxweiJBD8hBuFyfl9/HdyS+8R4uHkOZIjlMaDPjqosrqbQqinpoqywoYH3iVdNNkJE=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '6SAzJwWvNHxweiJBD8hBuFyfl9/HdyS+8R4uHkOZIjlMaDPjqosrqbQqinpoqywoYH3iVdNNkJE=',
   'x-amz-request-id': 'F6D1AA9745ADFEBE',
   'date': 'Fri, 14 Feb 2020 20:56:53 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Borrar el bucket

In [63]:
s3.Bucket(bucket_name).delete()

{'ResponseMetadata': {'RequestId': 'AD0D410932E0AF53',
  'HostId': 'uyoHxHQlhtK4SnDgxU0DPNGfrdg+8DOmLR4dda7bY9sLApMf/JwzXkHQEuaO5CgjxVK9vA5gHXY=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'uyoHxHQlhtK4SnDgxU0DPNGfrdg+8DOmLR4dda7bY9sLApMf/JwzXkHQEuaO5CgjxVK9vA5gHXY=',
   'x-amz-request-id': 'AD0D410932E0AF53',
   'date': 'Fri, 14 Feb 2020 20:57:52 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Referecias 

+ [S3 boto3 API Reference](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html)